In [1]:
from sklearn.linear_model import Lasso, Ridge, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import json
import pandas as pd
import numpy as np
import os
import sys
import pickle
import scipy.stats as stats 

# Using the new data cleaning code
First, we have to go from the current directory (inside `old-model-code/experiments/`) back to the base directory.

In [2]:
os.chdir('../../')


In [3]:
os.getcwd()

'/Users/xehu/Desktop/Team Process Mapping/tpm-horse-race-modeling'

In [4]:
DATASETFILEPATH = "./data_cache/multi_task_stageId_task_cumulative.pkl"

In [5]:
with open(DATASETFILEPATH, "rb") as horseracedataset_file:
    HorseRaceData = pickle.load(horseracedataset_file)

### Key attributes of the HorseRaceDataSet
- HorseRaceData.dvs
- HorseRaceData.composition_features
- HorseRaceData.size_feature
- HorseRaceData.task_features
- HorseRaceData.task_complexity_features
- HorseRaceData.conversation_features

Here's what I wanted before:

`team_composition_features`, `task_features`, `conv_features`, `targets`

In [6]:
targets = HorseRaceData.dvs
team_composition_features = pd.concat([HorseRaceData.size_feature, HorseRaceData.composition_features], axis = 1)
task_features = pd.concat([HorseRaceData.task_features, HorseRaceData.task_complexity_features], axis = 1)
conv_features = HorseRaceData.conversation_features

# Old Cleaning code (for comparison)

In [7]:
def drop_invariant_columns(df):
    """
    Certain features are invariant throughout the training data (e.g., the entire column is 0 throughout).

    These feature obviously won't be very useful predictors, so we drop them.
    
    This function works by identifying columns that only have 1 unique value throughout the entire column,
    and then dropping them.

    @df: the dataframe containing the features (this should be X).
    """
    nunique = df.nunique()
    cols_to_drop = nunique[nunique == 1].index
    return(df.drop(cols_to_drop, axis=1))

In [8]:
def read_and_preprocess_data(path, min_num_chats):
    conv_data  = pd.read_csv(path)

    # Filter this down to teams that have at least min_num of chats
    # Can also comment this out to re-run results on *all* conversations!
    conv_data = conv_data[conv_data["sum_num_messages"] >= min_num_chats]

    # Save the important information

    # DV
    dvs = conv_data[["score","speed","efficiency","raw_duration_min","default_duration_min"]]

    # Team Composition
    composition_colnames = ['birth_year', 'CRT', 'income_max', 'income_min', 'IRCS_GS', 'IRCS_GV', 'IRCS_IB', 'IRCS_IR',
                'IRCS_IV', 'IRCS_RS', 'political_fiscal', 'political_social', 'RME', 'country', 'education_level',
                'gender', 'marital_status', 'political_party', 'race', 'playerCount']
    
    # Select columns that contain the specified keywords
    composition = conv_data[[col for col in conv_data.columns if any(keyword in col for keyword in composition_colnames)]]

    # Task
    task = conv_data[['task', 'complexity']].copy()

    task_map_path = './features/task-mapping/task_map.csv' # get task map
    task_map = pd.read_csv(task_map_path)

    task_name_mapping = {
        "Moral Reasoning": "Moral Reasoning (Disciplinary Action Case)",
        "Wolf Goat Cabbage": "Wolf, goat and cabbage transfer",
        "Guess the Correlation": "Guessing the correlation",
        "Writing Story": "Writing story",
        "Room Assignment": "Room assignment task",
        "Allocating Resources": "Allocating resources to programs",
        "Divergent Association": "Divergent Association Task",
        "Word Construction": "Word construction from a subset of letters",
        "Whac a Mole": "Whac-A-Mole"
    }
    task.loc[:, 'task'] = task['task'].replace(task_name_mapping)
    task = pd.merge(left=task, right=task_map, on = "task", how='left')
    
    # Create dummy columns for 'complexity'
    complexity_dummies = pd.get_dummies(task['complexity'])
    task = pd.concat([task, complexity_dummies], axis=1)   
    task.drop(['complexity', 'task'], axis=1, inplace=True)

    # Conversation
    conversation = conv_data.drop(columns=list(dvs.columns) + list(composition.columns) + ['task', 'complexity', 'roundId', 'gameId', 'message', 'speaker_nickname', 'conversation_num', 'timestamp'])
    conversation = drop_invariant_columns(conversation) # drop invariant conv features

    return composition, task, conversation, dvs

In [9]:
multitask_cumulative_by_stage = 'conv/multi_task_stageId_cumulative_conv.csv'
multitask_cumulative_by_stage_and_task = 'conv/multi_task_stageId_task_cumulative_conv.csv'

In [10]:
# PARAMETERS
desired_target = "score"
data_path = "./data_cache/raw_output/"
min_num_chats = 0


In [11]:
team_composition_features_old, task_features_old, conv_features_old, targets_old = read_and_preprocess_data(data_path + multitask_cumulative_by_stage_and_task, min_num_chats=min_num_chats)

# Number of points in dataset
len(conv_features)

1043

## Compare outputs across different data analysis schemas

In [12]:
targets

,score,speed,efficiency,raw_duration_min,default_duration_min,task
0,85.040573,0.000556,0.047245,3.018050,3.0,Divergent Association
1,90.113825,0.000556,0.050063,3.016283,3.0,Divergent Association
2,60.000000,0.000333,0.020000,5.017700,5.0,Sudoku
3,49.000000,0.000333,0.016333,5.019833,5.0,Sudoku
4,100.000000,11.074630,1107.462975,4.446283,5.0,Sudoku
...,...,...,...,...,...,...
1038,100.000000,0.000333,0.033333,5.017133,5.0,Wolf Goat Cabbage
1039,90.000000,0.000333,0.030000,5.041683,5.0,Moral Reasoning
1040,90.000000,0.000333,0.030000,5.033550,5.0,Moral Reasoning
1041,0.000000,0.000333,0.000000,5.021850,5.0,Wolf Goat Cabbage


In [13]:
targets_old

,score,speed,efficiency,raw_duration_min,default_duration_min
0,85.040573,0.000556,0.047245,3.018050,3
1,90.113825,0.000556,0.050063,3.016283,3
2,60.000000,0.000333,0.020000,5.017700,5
3,49.000000,0.000333,0.016333,5.019833,5
4,100.000000,11.074630,1107.462975,4.446283,5
...,...,...,...,...,...
1038,100.000000,0.000333,0.033333,5.017133,5
1039,90.000000,0.000333,0.030000,5.041683,5
1040,90.000000,0.000333,0.030000,5.033550,5
1041,0.000000,0.000333,0.000000,5.021850,5


## Multi-Task Modeling Playground (our old modeling code)

In [15]:
len(conv_features)

1043

# Set up X's and y's

In [16]:
X_train = pd.concat([team_composition_features, task_features, conv_features], axis = 1)
y_train = targets

In [17]:
def columns_with_na(df):
    """
    Check and return columns that contain NaN (NA) values in a DataFrame.

    Parameters:
    - df: pandas DataFrame

    Returns:
    - List of column names with NaN values
    """
    # Check for NaN values in each column
    na_columns = df.columns[df.isna().any()].tolist()

    return na_columns

# Check columns that have NA
result = columns_with_na(X_train)
for colname in result:
    print(colname)

In [18]:
X_train

,playerCount,birth_year_mean,birth_year_std,CRT_mean,CRT_std,income_max_mean,income_max_std,income_min_mean,income_min_std,IRCS_GS_mean,...,Q5creativity_input_1,Q25_type6_mixed_motive,High,Low,Medium,PC1,PC2,PC3,PC4,PC5
0,3.0,1981.333333,2.081666,0.801587,0.216042,91665.666667,14433.756730,66666.666667,14433.756730,4.0000,...,0.758333,0,False,False,True,-8.925295,1.847588,-0.271020,3.978597,-1.682956
1,3.0,1981.333333,2.081666,0.801587,0.216042,91665.666667,14433.756730,66666.666667,14433.756730,4.0000,...,0.758333,0,False,True,False,-5.356021,0.031631,2.713015,4.840276,-3.583885
2,3.0,1981.333333,2.081666,0.801587,0.216042,91665.666667,14433.756730,66666.666667,14433.756730,4.0000,...,0.181818,0,False,False,True,-11.562210,1.357523,2.297750,-0.106859,-0.616964
3,3.0,1981.333333,2.081666,0.801587,0.216042,91665.666667,14433.756730,66666.666667,14433.756730,4.0000,...,0.181818,0,True,False,False,-7.214479,-2.965421,3.807253,0.513564,-0.855966
4,3.0,1981.333333,2.081666,0.801587,0.216042,91665.666667,14433.756730,66666.666667,14433.756730,4.0000,...,0.181818,0,False,True,False,-5.408882,-9.625545,3.044956,1.323865,-0.323487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,6.0,1984.250000,9.394147,0.714286,0.164957,38749.000000,24622.144504,25000.000000,17320.508076,3.9375,...,0.633333,0,False,False,True,17.611985,-4.714170,1.478930,2.920994,-5.765012
1039,6.0,1984.250000,9.394147,0.714286,0.164957,38749.000000,24622.144504,25000.000000,17320.508076,3.9375,...,0.404000,0,False,False,True,3.946101,7.638918,-15.634036,-6.505407,-6.029995
1040,6.0,1984.250000,9.394147,0.714286,0.164957,38749.000000,24622.144504,25000.000000,17320.508076,3.9375,...,0.404000,0,True,False,False,-8.576254,3.326463,-5.138322,-5.760661,-3.019051
1041,3.0,1970.333333,3.055050,0.214286,0.103783,23332.666667,15274.815885,13333.333333,15275.252317,4.0000,...,0.633333,0,False,True,False,-19.217539,10.925227,7.395674,3.735524,3.250466


## Try LASSO/Ridge Regression, one Set of Features at a Time

Here, we want to implement *leave-one-out cross-validation*, and use Q^2 as our metric.



Two updates to make here:

1. For nested LASSO/Ridge models, add the ability to initialize the model using the previous weights
2. Visualize importance using another library, like SHAP

In [19]:
# Note --- this uses k-fold cross-validation with k = 5 (the default)
# We are testing 10,000 different alphas, so I feel like this is an OK heuristic
def get_optimal_alpha(y_target, feature_columns_list, lasso):

    if(lasso == True):
        model = LassoCV(n_alphas = 10000)
        model.fit(X_train[feature_columns_list], y_train[y_target])
    else:
        model = RidgeCV(n_alphas = 10000)
        model.fit(X_train[feature_columns_list], y_train[y_target])
        
    return model.alpha_ # optimal alpha

In [20]:
def fit_regularized_linear_model(y_target, feature_columns_list, lasso=True, tune_alpha=False, prev_coefs = None, prev_alpha = None):

    if not tune_alpha:
        alpha = 1.0
    if (prev_alpha is not None):
        alpha = prev_alpha # use previous alpha
        print("Setting alpha to previous...")
        print(alpha)
    else:
        # Hyperparameter tune the alpha
        alpha = get_optimal_alpha(y_target, feature_columns_list, lasso=True)

    if lasso:
        model = Lasso(alpha=alpha)
    else:
        model = Ridge(alpha=alpha)

    if(prev_coefs is not None): # set weights to previous coefficients
        print("Setting coefficients ....")
        model.coef_ = prev_coefs

        print(model.coef_)

    # Calculation of Q^2 metric
    squared_model_prediction_errors = []
    squared_average_prediction_errors = []

    # Initialize a list to store coefficients
    coefficients_list = []

    # Leave one out -- iterate through the entire length of the dataset
    for i in range(len(y_train)):
        # Store the evaluation datapoint
        evaluation_X = X_train.iloc[[i]]
        evaluation_y = y_train.iloc[[i]][y_target]

        # Drop the ith datapoint (leave this one out)
        X_train_fold = X_train.drop(X_train.index[i])
        y_train_fold = y_train.drop(y_train.index[i])[y_target]

        # Fit the model
        model.fit(X_train_fold[feature_columns_list], y_train_fold)

        # Save the Prediction Error
        prediction = model.predict(evaluation_X[feature_columns_list])[0]
        squared_model_prediction_errors.append((evaluation_y - prediction) ** 2)

        # Save the Total Error for this fold
        squared_average_prediction_errors.append((evaluation_y - np.mean(y_train_fold)) ** 2)

        # Append the coefficients to the list
        coefficients_list.append(model.coef_)

    # Create a DataFrame with feature names as rows and iteration results as columns
    feature_coefficients = pd.DataFrame(coefficients_list, columns=feature_columns_list).T

    q_squared = 1 - (np.sum(squared_model_prediction_errors) / np.sum(squared_average_prediction_errors))
    print("Q^2: " + str(q_squared))

    return model, q_squared, feature_coefficients


In [21]:
def display_feature_coefficients(feature_coef_df):
    # Initialize a list to store DataFrames for each feature
    dfs = []

    # Iterate through the rows of the input DataFrame
    for feature_name, coefficients in feature_coef_df.iterrows():
        # Calculate the confidence interval without NaN values
        non_nan_coefficients = coefficients[~np.isnan(coefficients)]
        if len(non_nan_coefficients) == 0:
            # Handle the case where there are no valid coefficients
            continue

        mean_coef = non_nan_coefficients.mean()

        # Check if all coefficients in the row are the same
        if len(coefficients.unique()) == 1:
            # If all coefficients are the same, set the lower and upper CI to the mean
            confidence_interval = (mean_coef, mean_coef)
        else:
            std_error = non_nan_coefficients.sem()
            confidence_interval = stats.t.interval(0.95, len(non_nan_coefficients) - 1, loc=mean_coef, scale=std_error)

        # Create a DataFrame for the summary data
        temp_df = pd.DataFrame({
            "Feature": [feature_name],
            "Mean": [mean_coef],
            "Lower_CI": [confidence_interval[0]],
            "Upper_CI": [confidence_interval[1]]
        })

        # Append the temporary DataFrame to the list
        dfs.append(temp_df)

    # Concatenate all the DataFrames in the list into the final summary DataFrame
    summary_df = pd.concat(dfs, ignore_index=True)

    return summary_df

In [22]:
def sort_by_mean_abs(df):
    return df.reindex(df["Mean"].abs().sort_values(ascending=False).index)

In [23]:
# Go through the different types of features and fit models

# First, create a data structure that saves the result
result = {
    "model": [],
    "model_type": [],
    "features_included": [],
    "alpha": [],
    "q_squared": []
}

result_df = pd.DataFrame(result)

## Team composition features

In [24]:
len(team_composition_features.columns)

39

In [25]:
model_ridge_composition, mrc_q2, mrc_feature_coefficients = fit_regularized_linear_model(desired_target, team_composition_features.columns, lasso = False, tune_alpha = True)

result_df = pd.concat([result_df, pd.DataFrame({"model": [model_ridge_composition], "model_type": ["Ridge"], "features_included": ["Team Composition"], "alpha": [model_ridge_composition.alpha.round(4)], "q_squared": [mrc_q2]})], ignore_index=True)

Q^2: 0.01492438720730449


In [26]:
sort_by_mean_abs(display_feature_coefficients(mrc_feature_coefficients))

,Feature,Mean,Lower_CI,Upper_CI
25,RME_mean,0.430432,0.430189,0.430675
0,playerCount,0.140619,0.140438,0.140800
26,RME_std,-0.110888,-0.111105,-0.110672
13,IRCS_IB_mean,-0.095190,-0.095244,-0.095136
16,IRCS_IR_std,-0.075804,-0.075879,-0.075730
18,IRCS_IV_std,-0.073860,-0.073926,-0.073794
34,marital_status_std,-0.057417,-0.057466,-0.057367
20,IRCS_RS_std,0.052502,0.052443,0.052562
19,IRCS_RS_mean,-0.052150,-0.052226,-0.052074
31,gender_mean,-0.045907,-0.045952,-0.045862


In [27]:
model_lasso_composition, mlc_q2, mlc_feature_coefficients = fit_regularized_linear_model(desired_target, team_composition_features.columns, lasso = True, tune_alpha = True)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_lasso_composition], "model_type": ["Lasso"], "features_included": ["Team Composition"], "alpha": [model_lasso_composition.alpha.round(4)], "q_squared": [mlc_q2]})], ignore_index=True)

Q^2: 0.014807811977684548


## Task Features

In [28]:
len(task_features.columns)

27

In [29]:
model_ridge_task, mrt_q2, mrt_feature_coefficients = fit_regularized_linear_model(desired_target, task_features.columns, lasso = False, tune_alpha = True)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_ridge_task], "model_type": ["Ridge"], "features_included": ["Task Complexity"], "alpha": [model_ridge_task.alpha.round(4)], "q_squared": [mrt_q2]})], ignore_index=True)

Q^2: 0.23323504392160654


In [30]:
sort_by_mean_abs(display_feature_coefficients(mrt_feature_coefficients))

,Feature,Mean,Lower_CI,Upper_CI
22,Q5creativity_input_1,41.570133,41.564912,41.575354
3,Q6type_5_cc,-25.685501,-25.691578,-25.679425
6,Q9divisible_unitary,-25.012055,-25.015296,-25.008814
19,Q24eureka_question,-22.850365,-22.861137,-22.839593
8,Q11optimizing,20.191599,20.181947,20.201251
13,Q17within_sys_sol,-18.768273,-18.779743,-18.756802
9,Q13outcome_multip,18.133875,18.130229,18.137521
1,Q3type_1_planning,14.874848,14.865211,14.884485
7,Q10maximizing,-14.494884,-14.500190,-14.489579
12,Q16shared_knowledge,-14.078912,-14.080314,-14.077509


In [31]:
model_lasso_task, mlt_q2, mlt_feature_coefficients = fit_regularized_linear_model(desired_target, task_features.columns, lasso = True, tune_alpha = True)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_lasso_task], "model_type": ["Lasso"], "features_included": ["Task Complexity"], "alpha": [model_lasso_task.alpha.round(4)], "q_squared": [mlt_q2]})], ignore_index=True)

Q^2: 0.233850236072215


In [32]:
sort_by_mean_abs(display_feature_coefficients(mlt_feature_coefficients))

,Feature,Mean,Lower_CI,Upper_CI
22,Q5creativity_input_1,66.909022,66.895101,66.922942
6,Q9divisible_unitary,-44.936105,-44.957195,-44.915015
19,Q24eureka_question,-23.394759,-23.405378,-23.384141
9,Q13outcome_multip,21.259203,21.231599,21.286808
8,Q11optimizing,13.901990,13.868948,13.935032
5,Q8type_8_performance,8.648218,8.640361,8.656075
25,Low,4.599223,4.595660,4.602787
2,Q4type_2_generate,-2.599454,-2.617018,-2.581891
26,Medium,-2.226842,-2.230311,-2.223373
7,Q10maximizing,-0.055624,-0.066817,-0.044431


## Task + Composition Together

In [ ]:
# add together weights from previous models
previous_best_weights_ridge = np.array(list(model_ridge_composition.coef_) + list(model_ridge_task.coef_))

In [ ]:
task_comp_features = list(task_features.columns) + list(team_composition_features.columns)

model_ridge_taskcomp, mrtc_q2, mrtc_feature_coefficients = fit_regularized_linear_model(desired_target, task_comp_features, lasso = False, tune_alpha = False, prev_coefs = previous_best_weights_ridge, prev_alpha = model_ridge_task.alpha)

In [ ]:
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_ridge_taskcomp], "model_type": ["Ridge"], "features_included": ["Team Composition + Task Complexity"], "alpha": [model_ridge_taskcomp.alpha.round(4)], "q_squared": [mrtc_q2]})], ignore_index=True)

In [ ]:
directory = os.path.dirname(output_path + 'mrtc_feature_coefficients_initialized' + '.csv')
if not os.path.exists(directory):
    os.makedirs(directory)
mrtc_feature_coefficients.to_csv(output_path + 'mrtc_feature_coefficients_initialized' + '.csv')

In [ ]:
sort_by_mean_abs(display_feature_coefficients(mrtc_feature_coefficients))

In [ ]:
# add together weights from previous models
previous_best_weights_lasso = np.array(list(model_lasso_composition.coef_) + list(model_lasso_task.coef_))

In [ ]:
model_lasso_taskcomp, mltc_q2, mltc_feature_coefficients = fit_regularized_linear_model(desired_target, task_comp_features, lasso = True, tune_alpha = False, prev_coefs = previous_best_weights_lasso, prev_alpha = model_lasso_task.alpha)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_lasso_taskcomp], "model_type": ["Lasso"], "features_included": ["Team Composition + Task Complexity"], "alpha": [model_lasso_taskcomp.alpha.round(4)], "q_squared": [mltc_q2]})], ignore_index=True)

In [ ]:
directory = os.path.dirname(output_path + 'mltc_feature_coefficients_initialized' + '.csv')
if not os.path.exists(directory):
    os.makedirs(directory)
mltc_feature_coefficients.to_csv(output_path + 'mltc_feature_coefficients_initialized' + '.csv')

In [ ]:
sort_by_mean_abs(display_feature_coefficients(mltc_feature_coefficients))

## Conversation Alone

In [ ]:
model_lasso_comms, mlcom_q2, mlcom_feature_coefficients = fit_regularized_linear_model(desired_target, conv_features.columns, lasso = True, tune_alpha = True)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_lasso_comms], "model_type": ["Lasso"], "features_included": ["Communication"], "alpha": [model_lasso_comms.alpha.round(4)], "q_squared": [mlcom_q2]})], ignore_index=True)

In [ ]:
directory = os.path.dirname(output_path + 'mlcom_feature_coefficients' + '.csv')
if not os.path.exists(directory):
    os.makedirs(directory)
mlcom_feature_coefficients.to_csv(output_path + 'mlcom_feature_coefficients' + '.csv')

In [ ]:
sort_by_mean_abs(display_feature_coefficients(mlcom_feature_coefficients))

In [ ]:
model_ridge_comms, mrcom_q2, mrcom_feature_coefficients = fit_regularized_linear_model(desired_target, conv_features.columns, lasso = False, tune_alpha = True)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_ridge_comms], "model_type": ["Ridge"], "features_included": ["Communication"], "alpha": [model_ridge_comms.alpha.round(4)], "q_squared": [mrcom_q2]})], ignore_index=True)

In [ ]:
directory = os.path.dirname(output_path + 'mrcom_feature_coefficients' + '.csv')
if not os.path.exists(directory):
    os.makedirs(directory)
mrcom_feature_coefficients.to_csv(output_path + 'mrcom_feature_coefficients' + '.csv')

In [ ]:
sort_by_mean_abs(display_feature_coefficients(mrcom_feature_coefficients))

## Conversation Features + Task Features

In [ ]:
task_lasso_weights = np.array(list(model_lasso_task.coef_) + list(np.zeros(len((conv_features.columns)))))

In [ ]:
convtask_features = list(task_features.columns) + list(conv_features.columns)
model_lasso_tconv, mltconv_q2, mltconv_feature_coefficients = fit_regularized_linear_model(desired_target, convtask_features, lasso = True, tune_alpha = False, prev_coefs=task_lasso_weights, prev_alpha = model_lasso_task.alpha)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_lasso_tconv], "model_type": ["Lasso"], "features_included": ["Task Complexity + Communication"], "alpha": [model_lasso_tconv.alpha.round(4)], "q_squared": [mltconv_q2]})], ignore_index=True)

In [ ]:
directory = os.path.dirname(output_path + 'mltconv_feature_coefficients_initialized' + '.csv')
if not os.path.exists(directory):
    os.makedirs(directory)
mltconv_feature_coefficients.to_csv(output_path + 'mltconv_feature_coefficients_initialized' + '.csv')

In [ ]:
sort_by_mean_abs(display_feature_coefficients(mltconv_feature_coefficients))

In [ ]:
task_ridge_weights = np.array(list(model_ridge_task.coef_) + list(np.zeros(len((conv_features.columns)))))

In [ ]:
model_ridge_tconv, mrtconv_q2, mrtconv_feature_coefficients = fit_regularized_linear_model(desired_target, convtask_features, lasso = False, tune_alpha = False, prev_coefs=task_ridge_weights, prev_alpha = model_ridge_task.alpha)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_ridge_tconv], "model_type": ["Ridge"], "features_included": ["Task Complexity + Communication"], "alpha": [model_ridge_tconv.alpha.round(4)], "q_squared": [mrtconv_q2]})], ignore_index=True)

In [ ]:
directory = os.path.dirname(output_path + 'mrtconv_feature_coefficients_initialized' + '.csv')
if not os.path.exists(directory):
    os.makedirs(directory)
mrtconv_feature_coefficients.to_csv(output_path + 'mrtconv_feature_coefficients_initialized' + '.csv')

In [ ]:
sort_by_mean_abs(display_feature_coefficients(mrtconv_feature_coefficients))

## Model with All Features

In [ ]:
task_composition_lasso_weights = np.array(list(model_lasso_taskcomp.coef_) + list(np.zeros(len((conv_features.columns)))))

In [ ]:
all_features = list(task_features.columns) + list(team_composition_features.columns) + list(conv_features.columns)
model_lasso_all, mlall_q2, mlall_feature_coefficients = fit_regularized_linear_model(desired_target, all_features, lasso = True, tune_alpha = False, prev_coefs=task_composition_lasso_weights, prev_alpha = model_lasso_taskcomp.alpha)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_lasso_all], "model_type": ["Lasso"], "features_included": ["All Features"], "alpha": [model_lasso_all.alpha.round(4)], "q_squared": [mlall_q2]})], ignore_index=True)


In [ ]:
directory = os.path.dirname(output_path + 'mlall_feature_coefficients_initialized' + '.csv')
if not os.path.exists(directory):
    os.makedirs(directory)
mlall_feature_coefficients.to_csv(output_path + 'mlall_feature_coefficients_initialized' + '.csv')

In [ ]:
sort_by_mean_abs(display_feature_coefficients(mlall_feature_coefficients))

In [ ]:
task_composition_ridge_weights = np.array(list(model_ridge_taskcomp.coef_) + list(np.zeros(len((conv_features.columns)))))

model_ridge_all, mrall_q2, mrall_feature_coefficients = fit_regularized_linear_model(desired_target, all_features, lasso = False, tune_alpha = True, prev_coefs=task_composition_ridge_weights, prev_alpha = model_ridge_taskcomp.alpha)
result_df = pd.concat([result_df, pd.DataFrame({"model": [model_ridge_all], "model_type": ["Ridge"], "features_included": ["All Features"], "alpha": [model_ridge_all.alpha.round(4)], "q_squared": [mrall_q2]})], ignore_index=True)

In [ ]:
directory = os.path.dirname(output_path + 'mrall_feature_coefficients_initialized' + '.csv')
if not os.path.exists(directory):
    os.makedirs(directory)
mrall_feature_coefficients.to_csv(output_path + 'mrall_feature_coefficients_initialized' + '.csv')

In [ ]:
sort_by_mean_abs(display_feature_coefficients(mrall_feature_coefficients))

# Dataframe that summarizes all these experiments!

In [ ]:
result_df.sort_values(by = "q_squared", ascending = False)

In [ ]:
# Save for comparing results
directory = os.path.dirname(validation_results_output_name)
if not os.path.exists(directory):
    os.makedirs(directory)
result_df.sort_values(by = "q_squared", ascending = False).to_csv(validation_results_output_name)

# Feature Importance

In [ ]:
sort_by_mean_abs(display_feature_coefficients(mlall_feature_coefficients))

In [ ]:
def plot_top_n_features(data, n, filepath):
    # Calculate the absolute mean value and sort the DataFrame in descending order
    data['Absolute_Mean'] = data['Mean'].abs()
    top_n_features = data.sort_values(by='Absolute_Mean', ascending=False).head(n)

    # Define color mapping for the features
    color_map = {}
    name_map = {}
    for feature in task_features.columns:
        color_map[feature] = 'yellowgreen'
        name_map[feature] = "Task Feature"
    for feature in conv_features.columns:
        color_map[feature] = 'powderblue'
        name_map[feature] = "Conversation Feature"
    for feature in team_composition_features.columns:
        color_map[feature] = 'lightpink'
        name_map[feature] = "Team Composition Feature"

    # Create a horizontal bar graph
    plt.figure(figsize=(10, 6))

    handles = []

    for feature in top_n_features['Feature']:
        color = color_map.get(feature, 'k')  # Default to black if not in any list
        bars = plt.barh(feature, top_n_features[top_n_features['Feature'] == feature]['Mean'], color=color)
        handles.append(bars[0])

    # Customize the plot
    plt.xlabel('Mean Coefficient (Across LOO Cross Validation)', fontsize = 14)
    plt.title(f'Top {n} features for {desired_target} (min chats = {min_num_chats})', fontsize=20)
    plt.gca().invert_yaxis()  # Invert the y-axis to display the highest value at the top

    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    # Create a legend outside the plot area with unique labels
    unique_features = []
    unique_labels = []
    for feature in top_n_features['Feature']:
        if name_map.get(feature, feature) not in unique_labels:
            unique_labels.append(name_map.get(feature, feature))
            unique_features.append(feature)

    legend_handles = [plt.Line2D([0], [0], color=color_map.get(feature, 'k'), lw=4, label=name_map.get(feature, feature)) for feature in unique_features]
    plt.legend(handles=legend_handles, loc='center left', fontsize = 14, bbox_to_anchor=(1, 0.5))

    # Add labels to the bars with increased text size and Mean rounded to 2 decimals, consistently inside the bar
    label_offset = 0.4  # Adjust this value for proper spacing
    for bar, value, feature in zip(handles, top_n_features['Mean'], top_n_features['Feature']):
        label_x = (max(value, 0) if value >= 0 else min(value, 0))
        bbox = bar.get_bbox()
        label_y = bbox.bounds[1] + label_offset
        if value >= 0:
            plt.text(label_x, label_y, f'{value:.2f}', va='center', fontsize=12)
        else:
            plt.text(label_x, label_y, f'{value:.2f}', ha='right', va='center', fontsize=12)

    # Show the plot
    plt.savefig(filepath + ".svg")
    plt.savefig(filepath + ".png")
    plt.show()

In [ ]:
plot_top_n_features(display_feature_coefficients(mlall_feature_coefficients), 10, filepath = "./figures/multi_task_cumulative_stage" + "_" + desired_target + "_min_chat_num_" + str(min_num_chats))

Questions:
- More deeply understand difference between LASSO and Ridge
- Better understand `alpha` hyperparameter
- Why doesn't more features mean a better R^2? (Wouldn't the model 'throw out' features that don't work?)